In [1]:
%matplotlib inline
# A linha acima é necessária para visualizar os gráficos no jupyter notebook

import os
import pandas as pd
# Módulo com funções matemáticas para arrays multidimensionais (matrizes)
import numpy as np
# Módulo com funções de geração de gráficos
import matplotlib.pyplot as plt


In [46]:
# Carregando os dados
carne_produzida = pd.read_csv('meat-production-tonnes.csv')
pais_carne_produzida = carne_produzida['Entity']

carne_produzida.head(5)


,Entity,Code,Year,"Livestock Primary - Meat, Total - 1765 - Production - 5510 - tonnes (tonnes)"
0,Afghanistan,AFG,1961,129420
1,Afghanistan,AFG,1962,132206
2,Afghanistan,AFG,1963,138971
3,Afghanistan,AFG,1964,143830
4,Afghanistan,AFG,1965,150195


In [48]:
producao_2010 = carne_produzida['Year'] == 2010
carne_produzida.loc[producao_2010, :]

,Entity,Code,Year,"Livestock Primary - Meat, Total - 1765 - Production - 5510 - tonnes (tonnes)"
49,Afghanistan,AFG,2010,328160
107,Africa,NaN,2010,16638514
165,Albania,ALB,2010,91440
223,Algeria,DZA,2010,635153
281,American Samoa,ASM,2010,340
...,...,...,...,...
13289,Western Sahara,ESH,2010,2242
13347,World,OWID_WRL,2010,294619190
13405,Yemen,YEM,2010,323496
13494,Zambia,ZMB,2010,265615


In [61]:
target_consumo_per_capita = pd.read_csv('per-capita-meat-consumption-by-type-kilograms-per-year.csv')

producao_2010 = target_consumo_per_capita['Year'] == 2010
consumo_2010_per_cpt = target_consumo_per_capita.loc[producao_2010, :]
del consumo_2010_per_cpt['Other meats (kilograms per person)']
del consumo_2010_per_cpt['Poultry (kilograms per year)']
del consumo_2010_per_cpt['Pigmeat (kilograms per year)']
del consumo_2010_per_cpt['Mutton and Goat (kilograms per year)']

consumo_2010_per_cpt

,Entity,Code,Year,Beef and bufallo (kilograms per year)
49,Afghanistan,AFG,2010,4.70
106,Africa,NaN,2010,6.31
159,Albania,ALB,2010,21.38
216,Algeria,DZA,2010,4.60
273,Americas,NaN,2010,29.74
...,...,...,...,...
10769,Western Asia,NaN,2010,8.16
10822,World,OWID_WRL,2010,9.53
10875,Yemen,YEM,2010,4.96
10963,Zambia,ZMB,2010,5.64


In [102]:
target_IDH = pd.read_csv('HDI.csv')
del target_IDH['Unnamed: 0']
del target_IDH['Gross national income (GNI) per capita']
del target_IDH['GNI per capita rank minus HDI rank']
del target_IDH['Change in HDI rank 2010-2015']
del target_IDH['Average annual HDI growth 1990-2000']
del target_IDH['Average annual HDI growth 2000-2010']
del target_IDH['Average annual HDI growth 2010-2015']
del target_IDH['Change in HDI rank 2010-2015']

target_IDH

KeyError: 'Change in HDI rank 2010-2015'

In [99]:
desemprego_pais = pd.read_csv('API_ILO_country_YU.csv')

del desemprego_pais['Country Code']
del desemprego_pais['2011']
del desemprego_pais['2012']
del desemprego_pais['2013']
del desemprego_pais['2014']
desemprego_pais

,Country Name,2010
0,Afghanistan,20.600000
1,Angola,10.800000
2,Albania,25.799999
3,Arab World,25.022214
4,United Arab Emirates,9.800000
...,...,...
214,"Yemen, Rep.",30.299999
215,South Africa,50.799999
216,"Congo, Dem. Rep.",12.300000
217,Zambia,26.400000


In [115]:
pib = pd.read_csv('GDP.csv')
pib_2010 = pib.loc[:,['2010', 'Country']]


KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['Country'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"